Calculate mult for 64-to-4
inspired from scalability/scalability.ipynb

In [1]:
loadlevel = 0.5
numlink = 1066 
flowfile = "c2s_64_64_1_0_0_0"
bytespermult = 0
with open(f"/home/annzhou/DRing/src/emp/datacentre/flowfiles/{flowfile}",'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        bytespermult += int(tokens[2])
    
bytesperlink = 894784*1500*0.2 # 894784 packets per second, 1500B per packet, 200ms
totalbytes = bytesperlink*numlink # *2 for 2 directions, /2 for 2 hops per flow
usablebytes = totalbytes*loadlevel
mult = usablebytes/bytespermult
print(f"{flowfile}: {mult}")

c2s_64_64_1_0_0_0: 0.4547547394573216


Decide on what routing to use
inspired from computerouting2/computerouting2.ipynb, testspatial

In [4]:
import sys
import os
import gurobipy as gp
from gurobipy import GRB

def compute(graphfile,numsw,netpathfile,serverfile,flowfile,numserver):
    # read netpathfile
    netpath = list()
    for i in range(numsw):
        netpath.append(list())
        for j in range(numsw):
            netpath[i].append(list())
    with open(netpathfile,'r') as f:
        lines = f.readlines()
        # produce
        fromsw = 0
        tosw = 0
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
            else:
                path = [fromsw]
                tokens = line.split()
                for token in tokens:
                    hops = token.split("->")
                    path.append(int(hops[1]))
                netpath[fromsw][tosw].append(path)

        # check
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                numpaths = int(tokens[2])
                if len(netpath[fromsw][tosw])!=numpaths:
                    print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

    # read graphfile
    link = list()
    for i in range(numsw):
        link.append(list())
        for j in range(numsw):
            link[i].append(0)
    with open(graphfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split("->")
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            link[fromsw][tosw] = 1
            link[tosw][fromsw] = 1

    # read linkfailurefile (if needed)
    # if numfaillink > 0:
    #     with open(linkfailurefile,'r') as f:
    #         lines = f.readlines()
    #         for line in lines:
    #             tokens = line.split()
    #             fromsw = int(tokens[0])
    #             tosw = int(tokens[1])
    #             if link[fromsw][tosw] != 1:
    #                 print(f"ERROR: should have a link from {fromsw} to {tosw} but not")
    #             else:
    #                 link[fromsw][tosw] /= 2

    # read serverfile
    serverdict = dict()
    with open(serverfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split(',')
            serverdict[int(tokens[0])] = int(tokens[1])

    # read flowfile
    traffic = list()
    for i in range(numsw):
        traffic.append(list())
        for j in range(numsw):
            traffic[i].append(0)
    with open(flowfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split(",")
            fromsvr = int(tokens[0])
            tosvr = int(tokens[1])
            if fromsvr>=numserver or tosvr>=numserver: continue
            fromsw = serverdict[fromsvr]
            tosw = serverdict[tosvr]
            if fromsw == tosw: continue
            flowbytes = int(tokens[2])
            traffic[fromsw][tosw] += flowbytes


    # precompute
    flowsvialink = list()
    for i in range(numsw):
        flowsvialink.append(list())
        for j in range(numsw):
            flowsvialink[i].append(list())
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[fromsw][tosw] > 0:
                for pid,path in enumerate(netpath[fromsw][tosw]):
                    fidpidstr = f"{fromsw},{tosw},{pid}"
                    prevhop = fromsw
                    for hop in path[1:]:
                        flowsvialink[prevhop][hop].append(fidpidstr)
                        prevhop = hop

    for fromsw in range(numsw):
        for tosw in range(numsw):
            traffic[fromsw][tosw] /= 100000


    # Create a new model
    model = gp.Model("mcf")

    # Add variables
    maxpid = 0
    for fromsw in range(numsw):
        for tosw in range(numsw):
            maxpid = max(maxpid,len(netpath[fromsw][tosw]))
    vararr = list()
    for fromsw in range(numsw):
        vararr.append(list())
        for tosw in range(numsw):
            vararr[fromsw].append(list())
            for pid in range(maxpid):
                vararr[fromsw][tosw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    var = model.addVar(name=f"{fromsw}_{tosw}_{pid}")
                    vararr[fromsw][tosw][pid] = var
    k = model.addVar(name="k")

    # Set objective function
    model.setObjective(k, GRB.MAXIMIZE)

    # Add constraints
    # Constraint 0: for each fid: sum(pid) >= k * traffic[fid.from][fid.to]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[fromsw][tosw] > 0:
                varlist = list()
                for pid in range(len(netpath[fromsw][tosw])):
                    varlist.append(vararr[fromsw][tosw][pid])
                model.addConstr(sum(varlist)>=k*traffic[fromsw][tosw],f"c0_{fromsw}_{tosw}")

    # Constraint 1: for each link: sum(fid_pid) <= link[link.from][link.to]
    for linkfrom in range(numsw):
        for linkto in range(numsw):
            flowstrlist = flowsvialink[linkfrom][linkto]
            if len(flowstrlist) > 0:
                varlist = list()
                for flowstr in flowstrlist:
                    tokens = flowstr.split(',')
                    flowfrom = int(tokens[0])
                    flowto = int(tokens[1])
                    pid = int(tokens[2])
                    varlist.append(vararr[flowfrom][flowto][pid])
                model.addConstr(sum(varlist)<=link[linkfrom][linkto],f"c1_{linkfrom}_{linkto}")

    # Constraint 2: for each fid: for each pid: pid = prev_pid
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[fromsw][tosw] > 0:
                for pid in range(1,len(netpath[fromsw][tosw])):
                    model.addConstr(vararr[fromsw][tosw][pid]-vararr[fromsw][tosw][pid-1]==0,f"c2_{fromsw}_{tosw}_{pid}")

    # Optimize model
    model.setParam('Method',-1)
    model.setParam('Crossover',0)
    model.optimize()
    # modelfile = "/home/annzhou/DRing/src/emp/datacentre/experiments/torbottleneck/model.lp"
    # model.write(modelfile)

    # Print results
    if model.status == GRB.OPTIMAL:
        # print(f"Optimal objective value: {model.objVal}")
        resultfile = "/home/annzhou/DRing/src/emp/datacentre/experiments/networkbottleneck/whatrouting.txt"
        with open(resultfile,'a') as f:
            f.write(f"{flowfile},{routingname},{k.x}\n")
            # for fromsw in range(numsw):
            #     for tosw in range(numsw):
            #         if traffic[interval][fromsw][tosw] > 0:
            #             for pid in range(len(netpath[fromsw][tosw])):
            #                 var = vararr[fromsw][tosw][pid]
            #                 f.write(f"{var.varName},{var.x}\n")
    else:
        print("No optimal solution found")
    return

In [5]:
fileprefix = "/home/annzhou/DRing/src/emp/datacentre"
graphfile = f"{fileprefix}/graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist"
numsw = 80
numserver = 2988
routingnamearr = ["ecmp","su2","su3"]
serverfile = f"{fileprefix}/serverfiles/dring_2988_80_64"
clist = [64,48,32,16,4]
multlist = [[0,45,100],[0,80,100],[1,80,100],[7,20,100],[115,20,100]]
tseedlist = range(5)

for ic,c in enumerate(clist):
    mult = multlist[ic][0]
    numerator = multlist[ic][1]
    denominator = multlist[ic][2]
    for tseed in tseedlist:
        flowfile = f"{fileprefix}/flowfiles/c2s_{c}_{c}_{mult}_{numerator}_{denominator}_{tseed}"
        for routingname in routingnamearr:
            netpathfile = f"{fileprefix}/netpathfiles/netpath_{routingname}_dring.txt"
            compute(graphfile,numsw,netpathfile,serverfile,flowfile,numserver)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Set parameter Crossover to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 40 logical processors, using up to 20 threads

Optimize a model with 225792 rows, 223661 columns and 1298008 nonzeros
Model fingerprint: 0xcc508b39
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 219472 rows and 219472 columns
Presolve time: 2.84s
Presolved: 6320 rows, 4189 columns, 308020 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.08s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 1.279e+06
 Factor NZ  : 2.461e+06 (roughly 25 MB of memory)
 Factor Ops : 

Generate conffiles

In [7]:
graphname = "dring"
multlist = [[0,45,100],[0,80,100],[1,80,100],[7,20,100],[115,20,100]]
clist = [64,48,32,16,4]
# s=4
tseedlist = range(5)
sseed = 1
homedir = "/home/annzhou"

conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/networkbottleneck/networkbottleneck.conf"
makeleafspine = "MAKE"
makedring = "MAKE"
with open(conffile,'w') as f:
    for ic,c in enumerate(clist):
        s=c
        mult = multlist[ic][0]
        numerator = multlist[ic][1]
        denominator = multlist[ic][2]
        multstr = f"{mult}_{numerator}_{denominator}"
        for tseed in tseedlist:
            flowfilename = f"flowfiles/c2s_{c}_{s}_{multstr}_{tseed}"
            flowfile = f"{homedir}/DRing/src/emp/datacentre/{flowfilename}"

            routingdict = dict()
            with open(f"{homedir}/DRing/src/emp/datacentre/experiments/networkbottleneck/whatrouting.txt",'r') as fr:
                lines = fr.readlines()
                for line in lines:
                    tokens = line.split(',')
                    if tokens[0]==flowfile:
                        routingdict[tokens[1]] = float(tokens[2])
            if max(routingdict.values()) == routingdict["ecmp"]:
                routingname = "ecmp"
                rt1 = "ecmp"
                rt2 = 0
            elif max(routingdict.values()) == routingdict["su2"]:
                routingname = "su2"
                rt1 = "su"
                rt2 = 2
            elif max(routingdict.values()) == routingdict["su3"]:
                routingname = "su3"
                rt1 = "su"
                rt2 = 3
            
            lsqvarfile = f"qvarfiles/qvar_leafspine_0_0_ecmp_64"
            lsnetpathfile = f"netpathfiles/netpath_ecmp_leafspine.txt"
            dringqvarfile = f"qvarfiles/qvar_{graphname}_0_0_{routingname}_c2s_{c}_{s}_{tseed}_{multstr}_2_0_64"
            dringnetpathfile = f"netpathfiles/netpath_{routingname}_{graphname}.txt"
            dringgraphfile = f"graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist"
            dringserverfile = f"serverfiles/dring_2988_80_64"
        
            f.write(f"./run.sh LEAFSPINE {makeleafspine} 80 3072 64 leafspine_nb NEW_FILE null {mult} {numerator} {denominator} 0 0 0 0 0 0 ecmp 0 {flowfilename} null {sseed} {lsnetpathfile} {lsqvarfile} 64 50 150 200 0 0 > m_networkbottleneck_leafspine_{c}_{s}_{multstr}_{tseed}.log\n")
            f.write(f"./run.sh RRG {makedring} 80 2988 64 dring_nb NEW_FILE {dringgraphfile} {mult} {numerator} {denominator} 0 0 0 0 0 0 {rt1} {rt2} {flowfilename} {dringserverfile} {sseed} {dringnetpathfile} {dringqvarfile} 64 50 150 200 0 0 > m_networkbottleneck_dring_{c}_{s}_{multstr}_{tseed}.log\n")
            makeleafspine = "NOMAKE"
            makedring = "NOMAKE"